In [1]:
import sys
import os

# 프로젝트 루트 디렉토리 경로를 추가
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [2]:
import jax
import jax.numpy as jnp
import time
#disable jax JIT
#jax.config.update("jax_disable_jit", True)

from tqdm.autonotebook import trange
from functools import partial
from JAxtar.hash import hash_func_builder
from puzzle.slidepuzzle import SlidePuzzle
from heuristic.slidepuzzle_heuristic import SlidePuzzleHeuristic
from JAxtar.hash import HashTable

/tmp/ipykernel_321/2078866209.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


In [3]:
def _check_equal(state1, state2):
                tree_equal = jax.tree.map(lambda x, y: jnp.all(x == y), state1, state2)
                return jax.tree_util.tree_reduce(jnp.logical_and, tree_equal)

In [4]:
count = 1000
puzzle = SlidePuzzle(4)
hash_func = hash_func_builder(puzzle.State)
sample = jax.vmap(puzzle.get_initial_state)(key=jax.random.split(jax.random.PRNGKey(2),count))
new_sample = jax.vmap(puzzle.get_initial_state)(key=jax.random.split(jax.random.PRNGKey(1),count))
table = HashTable.build(puzzle.State, 1, int(1e4))

lookup = jax.jit(partial(HashTable.lookup, hash_func))
start = time.time()
idx, table_idx, found = jax.vmap(lookup, in_axes=(None, 0))(table, sample)
print(time.time()-start)

0.2036421298980713


In [5]:
batch = 4000
parallel_insert = jax.jit(partial(HashTable.parallel_insert, hash_func))
for i in range(18):
    sample = jax.vmap(puzzle.get_initial_state)(key=jax.random.split(jax.random.PRNGKey(i + 256),count))
    fidx, ftable_idx, old_found = jax.vmap(lookup, in_axes=(None, 0))(table, sample)
    batched_sample, filled = HashTable.make_batched(puzzle.State, sample, batch)
    start = time.time()
    table, inserted, _, _ = parallel_insert(table, batched_sample, filled)
    print(time.time()-start)
    idx, table_idx, found = jax.vmap(lookup, in_axes=(None, 0))(table, sample)
    for i in range(count):
        if found[i]:
            continue
        print(fidx[i], ftable_idx[i])
        print(table.table[fidx[i], ftable_idx[i]])
        print(idx[i], table_idx[i])
        print(sample[i])
        print(table.table[idx[i], table_idx[i]])
        s = sample[i]
        equal = jax.vmap(jax.vmap(_check_equal, in_axes=(0, None)), in_axes=(0, None))(table.table, s)
        args = jnp.argwhere(equal)
        arg_sample = args[0]
        print(*arg_sample)
        print(table.table[*arg_sample])
        assert jnp.any(equal), f"not found {s}" 
    print(jnp.mean(found), jnp.mean(old_found), jnp.mean(inserted))

0.8791465759277344
1.0 0.0 0.25
0.8592770099639893
1.0 0.0 0.25
0.0010271072387695312
1.0 0.0 0.25
0.0008397102355957031
1.0 0.0 0.25
0.0012676715850830078
1.0 0.0 0.25
0.0013384819030761719
1.0 0.0 0.25
0.0022971630096435547
1.0 0.0 0.25
0.0028181076049804688
1.0 0.0 0.25
0.003722667694091797
1.0 0.0 0.25
0.004606008529663086
1.0 0.0 0.25
0.0071184635162353516
1.0 0.0 0.25
0.0081024169921875
1.0 0.0 0.25
0.007685184478759766
1.0 0.0 0.25
0.00791621208190918
1.0 0.0 0.25
0.016243457794189453
1.0 0.0 0.25
0.013371467590332031
1.0 0.0 0.25
0.020327329635620117
1.0 0.0 0.25
0.024045944213867188
1.0 0.0 0.25


In [6]:
jnp.max(table.table_idx)

Array(2, dtype=uint8)

In [7]:
count = int(1e6)
puzzle = SlidePuzzle(4)
hash_func = hash_func_builder(puzzle.State)
sample = jax.vmap(puzzle.get_initial_state)(key=jax.random.split(jax.random.PRNGKey(2),count))
new_sample = jax.vmap(puzzle.get_initial_state)(key=jax.random.split(jax.random.PRNGKey(1),count))
table = HashTable.build(puzzle.State, 1, int(1e7))

lookup = jax.jit(partial(HashTable.lookup, hash_func))
start = time.time()
idx, table_idx, found = jax.vmap(lookup, in_axes=(None, 0))(table, sample)
print(time.time()-start)

0.20387935638427734


In [8]:
batch = 100000
parallel_insert = jax.jit(partial(HashTable.parallel_insert, hash_func))
for i in range(18):
    inserteds = []
    sample = jax.vmap(puzzle.get_initial_state)(key=jax.random.split(jax.random.PRNGKey(i + 256),count))
    idx, table_idx, found = jax.vmap(lookup, in_axes=(None, 0))(table, sample)
    same_ratio = jnp.mean(found)
    for j in trange(0, count, batch):
        table, inserted, _, table_idx = parallel_insert(table, sample[j:j+batch], jnp.ones(batch, dtype=jnp.bool_))
        inserteds.append(inserted)
    inserteds = jnp.concatenate(inserteds)
    idx, table_idx, found = jax.vmap(lookup, in_axes=(None, 0))(table, sample)
    print(jnp.mean(found), same_ratio, jnp.mean(inserteds))

  0%|          | 0/10 [00:00<?, ?it/s]

1.0 0.0 1.0


  0%|          | 0/10 [00:00<?, ?it/s]

1.0 0.0 1.0


  0%|          | 0/10 [00:00<?, ?it/s]

...

1.0 0.0 1.0


  0%|          | 0/10 [00:00<?, ?it/s]

1.0 2e-06 0.999998


  0%|          | 0/10 [00:00<?, ?it/s]

1.0 0.0 1.0


In [9]:
jnp.max(table.table_idx)

Array(2, dtype=uint8)

In [10]:
another_sample = jax.vmap(puzzle.get_initial_state)(key=jax.random.split(jax.random.PRNGKey(123),count))
idx, table_idx, found = jax.vmap(lookup, in_axes=(None, 0))(table, another_sample)
print(jnp.sum(found) / count)

1e-06


In [11]:
start = time.time()
inserteds = []
for i in trange(count // batch):
    table, inserted, _, _ = parallel_insert(table, another_sample[i*batch:(i+1)*batch], jnp.ones(batch, dtype=jnp.bool_))
    inserteds.append(inserted)
print(jnp.sum(jnp.concatenate(inserteds)) / count)
print("insert time:", time.time()-start)
start = time.time()
idx, table_idx, found = jax.vmap(lookup, in_axes=(None, 0))(table, sample)
print("check time:", time.time()-start)
print(jnp.sum(found) / count)
start = time.time()
idx, table_idx, found = jax.vmap(lookup, in_axes=(None, 0))(table, new_sample)
print("check time:", time.time()-start)
print(jnp.sum(found) / count)
idx, table_idx, found = jax.vmap(lookup, in_axes=(None, 0))(table, another_sample)
print("check time:", time.time()-start)
print(jnp.sum(found) / count)

  0%|          | 0/10 [00:00<?, ?it/s]

0.999999
insert time: 0.3193032741546631
check time: 0.06688785552978516
1.0
check time: 0.12557244300842285
1e-06
check time: 0.2550628185272217
1.0
